In [4]:
import numpy as np
import pandas as pd

from IPython.display import HTML
def display_df(df):
    return HTML(df.to_html())
    
from reinforcetrader.data import RawDataLoader
from reinforcetrader.features import FeatureBuilder

In [6]:
# Get S&P 500 raw data using Yahoo Finance API
data_loader = RawDataLoader(start_date='2000-02-20', end_date='2025-08-11')

# Download or load from cache the data
raw_data = data_loader.get_hist_prices()

# Inspect the raw S&P500 close and volume data
display_df(raw_data.head(10))

Loading cached data from data\raw\tickers_data_2000-02-20_2025-08-11.csv


In [7]:
# Instantiate Feature Builder to Build relevant features for state space
feature_builder = FeatureBuilder(hist_prices=raw_data)

# Build the features
feature_builder.build_features()
features_data = feature_builder.get_features()

# Inspect the features
display_df(features_data.head(10))

Building ticker features: 100%|███████████████████████████████████| 350/350 [00:39<00:00,  8.87it/s]


Features data saved to data\processed\tickers_features_2000-03-28_2025-08-08.csv
